# Computer Vision Business Recommendations
#### Luis Estrada, Michael Eugene, Maddie Hince, Kilmar Lazo, Cat Murad, and Elijah Soba

## Business Understanding

## Data Understanding

In [ ]:
# Import all necessary libraries
import sqlite3 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings

# Ignore warnings from library functions
warnings.filterwarnings('ignore')

# Set the style for plots 
sns.set_style('darkgrid')
sns.set_context('notebook')
sns.set()

We used data from three main sources to make recommendations for Computer Vision: 
- Box Office Mojo
- The Numbers
- IMDB (Internet Movie Database)

### Understanding Box Office Mojo

In [ ]:
# Define filepath & read in the data 
filepath = 'zippedData/bom.movie_gross.csv.gz'
movie_gross_df = pd.read_csv(filepath)

In [ ]:
# Take a preliminary look at the data 
movie_gross_df.head()

In [ ]:
# Take a look at the data frame information
movie_gross_df.info()

From looking at the information we can see immediately that there are a couple things wrong with the data:
- There are missing values in studio, domestic gross, and foreign gross
- Foreign gross is represented as an object instead of a float
- Year is represented as an integer, but it is probably better to have it as a datetime object

In [ ]:
# Lets see what percentage of the studio data is missing 
pct_missing_std = movie_gross_df.studio.isna().sum()/len(movie_gross_df.title)
print(f'Missing studio data represents {pct_missing_std*100}% of the data')

In [ ]:
# Lets see what percentage of the domestic gross data is missing 
pct_missing_dom = movie_gross_df.domestic_gross.isna().sum()/len(movie_gross_df.title)
print(f'Missing domestic gross data represents {pct_missing_dom*100}% of the data')

In [ ]:
# Lets see what percentage of the foreign gross data is missing 
pct_missing_for = movie_gross_df.foreign_gross.isna().sum()/len(movie_gross_df.title)
print(f'Missing foreign gross data represents {pct_missing_for*100}% of the data')

### Understanding The Numbers

In [ ]:
# Define filepath & read in the data 
filepath = 'zippedData/tn.movie_budgets.csv.gz'
budget_df = pd.read_csv(filepath)

In [ ]:
# Take a preliminary look at the data
budget_df.head()

In [ ]:
# Take a look at the data frame information
budget_df.info()

From looking at the information, we can see a few important things
- There are no missing (NaN) values
- All of the numerical columns are strings instead of floats

In [ ]:
# Lets see if there are any 0 values, which would be considered missing 
either_miss = len(budget_df[(budget_df['domestic_gross'] == '$0') | (budget_df['worldwide_gross'] == '$0')])
both_miss = len(budget_df[(budget_df['domestic_gross'] == '$0') & (budget_df['worldwide_gross'] == '$0')])
just_dom = len(budget_df[budget_df['domestic_gross'] == '$0'])
just_world = len(budget_df[budget_df['worldwide_gross'] == '$0'])
len_dataset = len(budget_df['domestic_gross'])
print(f'Number of missing gross values from either column {either_miss}')
print(f'Number of missing gross values from both columns {both_miss}')
print(f'Number of missing gross values from just domestic {just_dom}')
print(f'Number of missing gross values from just worldwide {just_world}')
print(f'Number of items in the entire dataset {len_dataset}')

In [ ]:
# Because there are numbers with 0 domestic and worldwide gross, lets see what fraction of the dataset it is
pct_0_dom = just_dom/len_dataset
print(f'Percentage of the data that is missing from the domestic gross column {pct_0_dom*100}')
pct_0_world = just_world/len_dataset
print(f'Percentage of the data that is missing from the worldwide gross column{pct_0_world*100}')

### Understanding IMDB

In [ ]:
# Establish a connection with SQL databasae and read in the data
filename = 'zippedData/im.db'
conn = sqlite3.connect(filename)
q = """SELECT *
       FROM movie_ratings
       JOIN movie_basics USING(movie_id)
       WHERE movie_basics.start_year >= 2010"""
ratings_df = pd.read_sql(q, conn)

In [ ]:
# Take a preliminary look at the data
ratings_df.head()

In [ ]:
# Take a look at the data frame information
ratings.info()

From looking at the information, we can see that there are some missing values in the runtime column and data types for columns are as expected

### Summary 
The data from Box Office Mojo included movie domestic and foreign gross, and movie production studio.
- Dataset contained in a .csv file
- Approximately 39% missing data in foreign gross column

Data from The Numbers included production budget and worldwide gross information.
- Dataset contained in a .csv file
- Approximately 6% missing data in worldwide gross

Data from IMDB included general movie information such as title, average ratings, and genres.
- Data represented as a relational SQL database
- Most analysis done using __movie_basics__ and __movie_ratings__ tables

We only considered movies in each dataset that had been released in 2010 or later.

## Data Preparation

### Preparing the Box Office Mojo Dataset

In [ ]:
# Lets recast foregin_gross as a float 
str_to_float = lambda x: float(str(x).replace(',', ''))
movie_gross_df['foreign_gross'] = movie_gross_df['foreign_gross'].map(str_to_float)

In [ ]:
# Lets test to see if the data type actually changed 
assert movie_gross_df['foreign_gross'].dtype  == np.float64

In [ ]:
# Lets replace missing studios with 'IND' representing independent
movie_gross_df['studio'] = movie_gross_df['studio'].fillna('IND')

In [ ]:
# Test to see if there are any missing values in the studio column
assert movie_gross_df['studio'].isna().sum() == 0

In [ ]:
# The missing % of data for domestic gross is small, so lets delete those rows
movie_gross_df = movie_gross_df.dropna(axis=0, subset=['domestic_gross'])

Because the amount of missing data from the foreign gross column is too much, we must impute the missing values. We impute missing values by calculating the median % change between foreign and domestic gross and using that to give an estimate of foreign gross as a function of domestic

In [ ]:
# That is too much, so lets find the median pct change between domestic and foreign films
med_pct_change = ((movie_gross_df['foreign_gross'] - movie_gross_df['domestic_gross'])/movie_gross_df['domestic_gross']).median()
print(med_pct_change)

0.6315266036797613


In [ ]:
# Now that we have the median % change, lets impute the missing foreign values 
impute_change = movie_gross_df['domestic_gross'] + (movie_gross_df['domestic_gross'] * med_pct_change)
movie_gross_df['foreign_gross'] = movie_gross_df['foreign_gross'].fillna(impute_change)

In [ ]:
# Test that we now have no more NaN values in the data 
assert movie_gross_df['foreign_gross'].isna().sum() == 0

In [ ]:
# Finally, test to see if there are any missing values anywhere in the data 
assert movie_gross_df.isna().sum().sum() == 0

In [ ]:
# Lets normalize the data to millions to make numbers more manageable
movie_gross_df['total_gross'] = movie_gross_df['total_gross']/1000000
movie_gross_df['domestic_gross'] = movie_gross_df['domestic_gross']/1000000
movie_gross_df['foreign_gross'] = movie_gross_df['foreign_gross']/1000000

In [ ]:
# Lets change the title to avoid any issues with spacing
movie_gross_df['title'] = movie_gross_df['title'].map(lambda x: x.strip().lower().replace(" ", ""))

### Preparing the Numbers Dataset

In [ ]:
# Lets define a function that will parse through a string of $xxx,xxx,xxx and turn it into a float
money_to_float = lambda x: float(x.replace('$','').replace(',',''))

In [ ]:
# Lets test to make sure this function is behaving the way that we anticipate
assert money_to_float('$1,000') == 1000

In [ ]:
# Update the dtype of the production_budget column 
budget_df['production_budget'] = budget_df['production_budget'].map(money_to_float)

In [ ]:
# Update the dtype of the domestic_gross column 
budget_df['domestic_gross'] = budget_df['domestic_gross'].map(money_to_float)

In [ ]:
# Update the dtype of the worldwide_gross column 
budget_df['worldwide_gross'] = budget_df['worldwide_gross'].map(money_to_float)

In [ ]:
# Test to make sure it worked
assert budget_df['production_budget'].dtype == np.float64

In [ ]:
# Lets make the release date an int dtype that only has the year
budget_df['release_date'] = pd.to_datetime(budget_df['release_date']).dt.year

In [ ]:
# Test to make sure the dtype is now correct
assert budget_df['release_date'].dtype == np.int64

In [ ]:
# < 10% of data is $0, so lets drop those records
budget_df = budget_df[budget_df['domestic_gross'] != 0]

In [ ]:
# Test to make sure we dropped correctly
assert len(budget_df[budget_df['domestic_gross'] == 0]) == 0
assert len(budget_df[budget_df['worldwide_gross'] == 0]) == 0

In [ ]:
# Finally, we can normalize monetary columns to the millions 
budget_df['production_budget'] = budget_df['production_budget']/1000000
budget_df['domestic_gross'] = budget_df['domestic_gross']/1000000
budget_df['worldwide_gross'] = budget_df['worldwide_gross']/1000000

### Preparing the IMDB Dataset

In [ ]:
# For the sake of our analysis, we can drop unnecessary columns 
to_drop = ['movie_id', 'original_title', 'start_year', 'runtime_minutes']
ratings_df = rateings_df.drop(labels=to_drop, axis=1)

In [ ]:
# Lets change the titles to avoid any conflicts with spacings
rate_genre_df['primary_title'] = rate_genre_df['primary_title'].map(lambda x: x.strip().lower().replace(" ", ""))

In [ ]:
# Lets filter out any movies that have a bottom 25% number of votes 
ratings_df = ratings_df[ratings_df['numvotes'] >= 62]

## Data Analysis & Visualizations
(After every visualization that corresponds to our recommendation, mention in **bold** the recommendation that we make)

### Box Office Mojo Analysis

### The Numbers Analysis

### IMDB Analysis 
(Include IMDB combined with Box Office Mojo data analysis here too)

### Highest-Rated Genres

### Budget-to-Box Office Ratio

### Highest-Grossing Studios

## Statistical Communication

After identifying __animation__ as the genre Computer Vision should pursue, we wanted to explore the difference between the genre's foreign and domestic gross. 

We hypothesized that:
- A movie in the animation category will gross higher internationally than domestically. 

Based on the results of our t-test, we can determine where to allocate more marketing efforts to maximize profitability.

### T-test
We want to know if __animation__ movies gross more internationally or domestically. 
- We chose the __animation__ genre because it is one of the genres with the highest average ratings, and appears as a genre in highly-grossing films from the highest-grossing studio (BV Studios, which turns out to be Disney/Pixar).

Our __null hypothesis__ is that the animation genre grosses the same internationally and domestically.

Our __alternative__ hypothesis is that animation movies gross more internationally than domestically. 

In [ ]:
# First, lets get the dataframe of only animation movies 
genre_to_grab = "Animation"

ttest_df = combined[combined['genres'].str.contains(genre_to_grab)][['domestic_gross', 'foreign_gross']]


In [ ]:
# Lets check to see if we have enough samples
len(ttest_df)

159

In [ ]:
# Lets look at the statistics to give us a basis for our intution
ttest_df.describe()

,domestic_gross,foreign_gross
count,159.000000,159.000000
mean,89.131074,164.672467
std,111.552661,199.679024
min,0.000600,0.000979
25%,0.889000,9.781427
50%,48.000000,75.900000
75%,145.950000,276.900000
max,608.600000,875.700000


In [ ]:
# Lets sample from our data
domestic_sample = ttest_df['domestic_gross'].sample(100)
foreign_sample = ttest_df['foreign_gross'].sample(100)

In [ ]:
# Lets perform a one tailed two sample t test
import scipy.stats as stats
alpha = .05
tstat, pvalue = stats.ttest_ind(domestic_sample, foreign_sample, equal_var=False)

answer = 'Reject null hypothesis' if pvalue < alpha else 'Fail to reject null hypothesis'
print(answer)

Reject null hypothesis


In [ ]:
# Plot the t statistic on a t distribution
tval_alpha = stats.t.ppf(.05, 99)
x = np.linspace(-5,5,1000)
y = stats.t.pdf(x, 99)
fig, axs = plt.subplots()
axs.plot(x,y)
axs.axvline(tstat)
axs.axvline(tval_alpha)
axs.axvspan(-5, tval_alpha, alpha=0.5, color='red')
plt.text(tval_alpha + .1 ,.3,'reject region',rotation=90)
plt.text(tstat + .1 ,.3,'t statistic',rotation=90)
axs.set_title('T-Distribution w/D.F = 99')

The results of our statistical test show that with 95% confidence we can conclude foreign gross is significantly greater than domestic gross regarding movies with the __animation__ genre. 
- This indicates that more money should be spent marketing to foreign markets as opposed to domestic.

## Conclusion
(Emphasize three recommendations here )